# <p style="text-align: center;"> AP EKF yaw angle and bias estimation

In [25]:
import sympy as sp
import numpy as np
from scipy import linalg
from sympy import *
from IPython.display import Math, Latex

## List of tuning parameters for yaw fusion
#### A. Process noise parameters
1. $GYRO\_P\_NSE$ : Rate gyro noise [rad/s] $(AP \; value: 0.03)$ <br>
&emsp; <b>AP definition:</b> This control disturbance noise controls the growth of estimated error due to gyro measurement errors excluding bias. Increasing it makes the flter trust the gyro measurements less and other measurements more. <br>
<font color = red> <i><b>Interpretation:</b> This parameter dictates the amount of deviation from the motion model (of angular velocities) that the estimated model (of angular velocities) can have, i.e., whether the estimated model (of angular velocities) will follow the motion model (of angular velocities) or the true values (of angular velocities).</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> The estimated model (of angular velocities) will follow the true model (of angular velocities) and will be more sensitive to noisy measurements.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> The estimated model (of angular velocities) will follow the motion model (of angular velocities), will be less sensitive to noisy measurements, and rapid changes in the true model (of angular velocities) will be ignored (which should actually be tracked.).</i> </font> <br>
</br>
2. $GBIAS\_P\_NSE$ : Rate gyro bias stability [rad/s/s] $(AP \; value: 0.0005)$ <br>
&emsp; <b>AP definition:</b> This state  process noise controls growth of the gyro delta angle bias state error estimate. Increasing it makes rate gyro bias estimation faster and noisier. <br>
<font color = red> <i><b>Interpretation:</b> This parameter dictates the amount of deviation from the motion model (of angular velocities bias) that the estimated model (of angular velocities bias) can have, i.e., whether the estimated model (of angular velocities bias) will follow the motion model (of angular velocities bias) or the true values (of angular velocities bias).</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> The estimated model (of angular velocities bias) will follow the true model (of angular velocities bias) and will be more sensitive to noisy measurements.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> The estimated model (of angular velocities bias) will follow the motion model (of angular velocities bias), will be less sensitive to noisy measurements, and rapid changes in the true model (of angular velocities bias) will be ignored (which should actually be tracked.).</i> </font>

#### B. Measurement uncertainty parameters
3. $YAW\_M\_NSE$ : Yaw measurement noise [rad] $(AP \; value: 0.5)$ <br>
&emsp; <b>AP definition:</b> This is the RMS value of noise in yaw measurements from the magnetometer. Increasing it reduces the weighting on these measurements. <br>
<font color = red> <i><b>Interpretation:</b> This parameter denotes the yaw measurement accuracy of the gyroscope. It dictates whether the estimated model (of yaw) will follow the motion model (of yaw) or the true measurements (of yaw).</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> The estimated model (of yaw) will favor the motion model (of yaw) and rapid changes in the true measurements (of yaw) will be ignored.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> The estimated model (of yaw) will favor the true measurements (of yaw).</i> </font> <br>
</br>
4. $MAG\_M\_NSE$ : Magnetometer measurement noise [Gauss] $(AP \; value: 0.05)$ <br>
&emsp; <b>AP definition:</b> This is the RMS value of noise in magnetometer measurements. Increasing it reduces the weighting on these measurements. <br>
<font color = red> <i><b>Interpretation:</b> This parameter denotes the yaw measurement accuracy of the compass. It dictates whether the estimated model (of yaw) will follow the motion model (of yaw) or the true measurements (of yaw).</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> The estimated model (of yaw) will favor the motion model (of yaw) and rapid changes in the true measurements (of yaw) will be ignored.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> The estimated model (of yaw) will favor the true measurements (of yaw).</i> </font>

#### C. Innovation consistency check parameters
5. $YAW\_I\_GATE$ : Yaw measurement gate size [%] $(AP \; value: 300)$ <br>
&emsp; <b>AP definition:</b> This sets the percentage number of standard deviations applied to the magnetometer yaw measurement innovation consistency check. Decreasing it makes it more likely that good measurements will be rejected. Increasing it makes it more likely that bad measurements will be accepted. <br>
<font color = red> <i><b>Interpretation:</b> This parameter sets the margin within which if the difference between the estimated and measured yaw falls, fusion of yaw will be successful.</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> The margin will be wider making it possible that even larger differences between the estimated and measured yaw are accepted for fusion. Bad measurements may be accepted in this way.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> The margin will be narrower, allowing only small differences between the estimated and measured yaw to be accepted for fusion. Good measurements may be rejected in this way.</i> </font> <br>
</br>
6. $FS\_EKF\_THRESH$ : EKF failsafe variance threshold $(AP \; value: 0.8)$ <br>
&emsp; <b>AP definition:</b> Allows setting the maximum acceptable compass, position, and velocity variance. <br>
<font color = red> <i><b>Interpretation:</b> This parameter can be thought of as the RMS value of the innovations of yaw. While the GATE parameter defines a margin to accept or reject (fluctuating) measurements, the FS_EKF_THRESH parameter defines a maximum acceptable limit on the stable or RMS values of the innovations, exceeding which will result in an EKF failsafe action. A balance has to be struck between choosing the two parameters so as to satisfy the margin as well as the limit.</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> In the worst case scenario, having a high GATE parameter (s.t. for e.g. posTestRatio $<$ 1) and also increasing the FS_EKF_THRESH parameter will result in bad measurements with high RMS values to be accepted, therefore resulting in a reduced estimation accuracy.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> In the worst case scenario, having a low GATE parameter (s.t. for e.g. posTestRatio $<$ 1) and also decreasing the FS_EKF_THRESH parameter will result in good measurements with low RMS values to be rejected, therefore resulting in an EKF failsafe action.</i> </font>

## System model

The system model is represented in the form of the state extrapolation equation: <br>
<p style="text-align: center;"> $X_{t|t-1} = F.X_{t-1|t-1} + G.U_{t-1|t-1}$ <br> </p>

where, <br>
$X_{t|t-1}$ = Predicted system state at time t <br>
$F$ = State transition matrix <br>
$X_{t-1|t-1}$ = System state at time t-1 <br>
$G$ = Control matrix <br>
$U_{t-1|t-1}$ = Control vector at time t-1

In [3]:
# XT_predict_compact = Predicted system state at time t
# F = State transition matrix
# XT_previous = System state at time t-1
# G = Control matrix
# U = Control vector at time t-1

yaw_t1, yaw_bias_t1, yaw_t, yaw_rate_t, yaw_bias_t, delt = sp.symbols('\Psi_{t|t-1}, \Psi_{b|t|t-1}, \Psi_{t-1}, \dot{\Psi}_{t-1}, \Psi_{b|t-1}, \Delta{t}')

Xt_predict_compact = sp.Matrix([[yaw_t1],[yaw_bias_t1]])
F = sp.Matrix([[1,0],[0,1]])
Xt_previous = sp.Matrix([[yaw_t],[yaw_bias_t]])
G = sp.Matrix([[delt],[0]])
Ut_previous = sp.Matrix([[yaw_rate_t]])

display(Math(r'X_{{t|t-1}} = {}, F = {}, X_{{t-1|t-1}} = {}, G = {}, U_{{t-1|t-1}} = {}'.format(sp.latex(Xt_predict_compact), sp.latex(F), sp.latex(Xt_previous), sp.latex(G), sp.latex(Ut_previous))))

<IPython.core.display.Math object>

Arranging the matrices in the form of the state extrapolation equation: $X_{t|t-1} = F.X_{t-1|t-1} + G.U_{t-1|t-1}$

In [4]:
display(Math(r'{} = {}{} + {}{}'.format(sp.latex(Xt_predict_compact), sp.latex(F), sp.latex(Xt_previous), sp.latex(G), sp.latex(Ut_previous))))

<IPython.core.display.Math object>

## Predicted system state $X$ (at time t-1, for time t)

In [5]:
Xt_predict = F*Xt_previous+G*Ut_previous

display(Math(r'X_{{t|t-1}} = {} = {}'.format(sp.latex(Xt_predict_compact), sp.latex(Xt_predict))))

<IPython.core.display.Math object>

## Process noise

In [6]:
# Q = Process noise covariance matrix
# q = Process noise standard deviation (sigma) of each state variable

Q_yaw, Q_yaw_bias = sp.symbols('q_{\Psi}^2, q_{{\Psi}|b}^2')

Q = sp.Matrix([[Q_yaw,0,],[0,Q_yaw_bias]])

display(Math(r'Q = {}'.format(sp.latex(Q))))

<IPython.core.display.Math object>

The process noise covariance matrix and its variables in the AP code are as follows:

In [7]:
Q_yaw_AP, Q_yaw_bias_AP = sp.symbols('{(gyrNoise*\Delta{t})}^2, {(gyroBiasProcessNoise*(\Delta{t})^2)}^2')

Q_AP = sp.Matrix([[Q_yaw_AP,0],[0,Q_yaw_bias_AP]])

display(Math(r'Q_{{AP}} = {}'.format(sp.latex(Q_AP))))

<IPython.core.display.Math object>

$gyrNoise = GYRO\_P\_NSE$ <br>
$gyroBiasProcessNoise = GBIAS\_P\_NSE$ <br>

## Measurement uncertainty

In [8]:
# R = Measurement uncertainty covariance matrix
# r = Measurement uncertainty standard deviation (sigma) of each sensor that can measure a system state variable

R_yaw, R_yaw_rate = sp.symbols('r_{\Psi}^2, r_{\dot{\Psi}}^2')

R = sp.Matrix([[R_yaw,0],[0,R_yaw_rate]])

display(Math(r'R = {}'.format(sp.latex(R))))

<IPython.core.display.Math object>

The measurement uncertainty covariance matrix and its variables in the AP code are as follows:

In [24]:
R_yaw_AP, R_yaw_rate_AP = sp.symbols('{(yawAngDataDelayed.yawAngErr)}^2, {(yawAngDataDelayed.yawAngErr/\Delta{t})}^2')

R_AP = sp.Matrix([[R_yaw_AP,0],[0,R_yaw_rate_AP]])

display(Math(r'R_{{AP}} = {}'.format(sp.latex(R_AP))))

<IPython.core.display.Math object>

#### Note on $R_{AP}$:
If yawFusionMethod = GPS <br>
&emsp; $R_{AP|11} = (yawAngDataDelayed.yawAngErr)^2 = (yawNoise)^2 = (YAW\_M\_NSE)^2$ <br>
Else <br>
If yawFusionMethod = GSF <br>
&emsp; $R_{AP|11} = gsfYawVariance$ <br>
Else <br>
If yawFusionMethod = Static <br>
&emsp; $R_{AP|11} = (yawAngDataStatic.yawAngErr)^2 = (yawNoise)^2 = (YAW\_M\_NSE)^2$ <br>
Else <br>
If yawFusionMethod = Magnetometer <br>
&emsp; $R_{AP|11} = (magNoise)^2 = (MAG\_M\_NSE)^2$ <br>
Else <br>
If yawFusionMethod = Predicted <br>
&emsp; $R_{AP|11} = (yawNoise)^2 = (MAG\_M\_NSE)^2$

## Initialized estimation uncertainty covariance matrix $P$ (at time t-1, for time t-1)

In [10]:
# Pt_previous = Estimated estimation uncertainty covariance matrix of the system state variables at time t-1

P_yaw, P_yaw_bias = sp.symbols('P_{{\Psi}|t-1|t-1}, P_{{\Psi}|b|t-1|t-1}')

Pt_previous = sp.Matrix([[P_yaw,0],[0,P_yaw_bias]])

display(Math(r'P_{{t-1|t-1}} = {}'.format(sp.latex(Pt_previous))))

<IPython.core.display.Math object>

## Predicted estimation uncertainty covariance matrix $P$ (at time t-1, for time t)

In [11]:
# Pt_predict = Predicted estimation uncertainty covariance matrix of the system state variables at time t

Pt_predict = (F*Pt_previous*F.T + Q)

display(Math(r'P_{{t|t-1}} = {}'.format(sp.latex(Pt_predict))))

<IPython.core.display.Math object>

## Updated measurement vector (at time t, for time t)

In [12]:
# Zt_update = Measurement vector at time t containing the sensor measurements
# H = Observation matrix to map the measurement vector to the system state variables

yaw_t, yaw_rate_t = sp.symbols('z_{{\Psi}|t}, z_{\dot{\Psi}|t}')

Zt_update = sp.Matrix([[yaw_t],[yaw_rate_t]])
H = sp.Matrix([[1,0],[0,0]])

display(Math(r'Z_{{t|t}} = {}, H = {}'.format(sp.latex(Zt_update), sp.latex(H))))

<IPython.core.display.Math object>

The measurement vector and its variables in the AP code are as follows:

In [13]:
yaw_t_AP, yaw_rate_t_AP = sp.symbols('yawAngDataDelayed.yawAng, yawAngDataDelayed.yawAng/imuDataDelayed.delAngDT')

Zt_update_AP = sp.Matrix([[yaw_t_AP],[yaw_rate_t_AP]])
display(Math(r'Z_{{t|t|AP}} = {}'.format(sp.latex(Zt_update_AP))))

<IPython.core.display.Math object>

#### Note on $Z_{t|t|AP}$:
If yawFusionMethod = GPS <br>
&emsp; $Z_{t|t|AP|11} = yawAngDataDelayed.yawAng$ <br>
Else <br>
If yawFusionMethod = GSF <br>
&emsp; $Z_{t|t|AP|11} = gsfYaw$ <br>
Else <br>
If yawFusionMethod = Static <br>
&emsp; $Z_{t|t|AP|11} = yawAngDataStatic.yawAng$ <br>
Else <br>
If yawFusionMethod = Magnetometer <br>
&emsp; $Z_{t|t|AP|11} = yawAngMeasured$ <br>
Else <br>
If yawFusionMethod = Predicted <br>
&emsp; $Z_{t|t|AP|11} = not \; mentioned \; in \; AP \; code$

## Updated innovation matrix (at time t, for time t)

In [14]:
# Yt_update = Updated innovation matrix at time t

Yt_update = H*Xt_predict_compact - Zt_update

display(Math(r'Y_{{t|t}} = {}'.format(sp.latex(Yt_update))))

<IPython.core.display.Math object>

## Updated innovation covariance matrix (at time t, for time t)

In [15]:
# St_update = Updated innovation covariance matrix at time t

St_update = H*Pt_predict*H.T + R

display(Math(r'S_{{t|t}} = {}'.format(sp.latex(St_update))))

<IPython.core.display.Math object>

## Innovation consistency check for magnetometer health status

In [26]:
Yt_11, maxYawInnovYaw = sp.symbols('Y_{t|t|11}, maxYawInnovYaw')

display(Math(r'maxYawInnovYaw = max(0.01*yawInnovGate, 1)^2 * S_{{t|t|11}}'))
display(Math(r'yawInnovGate = YAW\_I\_GATE'))

yawTestRatio = (Yt_11)**2 / maxYawInnovYaw

display(Math(r'yawTestRatio = {}'.format(sp.latex(yawTestRatio))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### Condition for successful fusion of magnetometer yaw data:
If (yawTestRatio < 1) <br>
&emsp; magHealth = true; \\\ can fuse mag yaw data <br>
Else <br>
&emsp; magHealth = false; \\\ cannot fuse mag yaw data

#### EKF failsafe due to yaw variance
If $\sqrt{yawTestRatio} > FS\_EKF\_THRESH$ for two consecutive variances, an EKF failsafe will be triggered (1. Land; 2. AltHold; 3. Land even in stabilize).

In [16]:
Num = sp.adjoint(St_update).simplify()

In [17]:
Den = sp.det(St_update).simplify()

In [18]:
St_inv = (Num/Den).simplify()

display(Math(r'S_{{t|t}}^{{-1}} = {}'.format(sp.latex(St_inv))))

<IPython.core.display.Math object>

## Updated Kalman gain $K$ (at time t, for time t)

In [19]:
# Kt_update = Updated Kalman gain at time t

Kt_update = (Pt_predict*H.T*St_inv).simplify()

display(Math(r'K_{{t|t}} = {}'.format(sp.latex(Kt_update))))

<IPython.core.display.Math object>

## Estimated system state $X$ (at time t, for time t)

In [20]:
# Xt_estimate = Estimated system state at time t

Xt_estimate = (Xt_predict_compact + Kt_update*(Yt_update))

display(Math(r'X_{{t|t}} = {}'.format(sp.latex(Xt_estimate))))

<IPython.core.display.Math object>

## Estimated estimation uncertainty covariance matrix $P$ (at time t, for time t)

In [21]:
# Pt_estimate = Estimated estimation uncertainty matrix of the system state variables at time t

I = sp.Matrix([[1,0],[0,1]])

Pt_estimate = ((I - Kt_update*H)*Pt_predict)

display(Math(r'P_{{t|t}} = {}'.format(sp.latex(Pt_estimate))))

<IPython.core.display.Math object>